In [38]:
import re
import math
import warnings
import tqdm
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from sympy import symbols, Eq, solve
from matplotlib.pyplot import MultipleLocator
from sklearn.preprocessing import MinMaxScaler, Binarizer
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
warnings.filterwarnings("ignore")


%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

In [39]:
# data1=pd.read_excel('/root/GDUT/表1-患者列表及临床信息.xlsx')
# data2=pd.read_excel('/root/GDUT/表2-患者影像信息血肿及水肿的体积及位置.xlsx')
# data3=pd.read_excel('/root/GDUT/表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx')
# data4=pd.read_excel('/root/GDUT/表4-答案文件.xlsx')
data5=pd.read_excel('2b聚类数据.xlsx')
data6=pd.read_excel('2a数据_已替换时间戳_已矫正.xlsx')

# data1.rename(columns={data1.columns[0]: "ID"}, inplace=True)
# data2.rename(columns={data2.columns[0]: "ID"}, inplace=True)

# data_f_1=pd.read_excel('附表1-检索表格-流水号vs时间.xlsx')
# data_f_time=pd.read_excel('时间点.xlsx')

In [40]:
# data6['ID'] = data5['ID'].head(100)
data7 = data5.head(100)

# 转换独热编码
xueya_list = []
for i in data7['血压']:
    y1 = i.split('/')[0]
    y2 = i.split('/')[1]
    if int(y1) > 139 or int(y1) < 90:
        status_i = 0
    elif int(y2) > 89 or int(y2) < 60:
        status_i = 0
    else:
        status_i = 1
    xueya_list.append(status_i)
data7['血压'] = np.array(xueya_list)
data7['性别_男'] = data5['性别'].apply(lambda x: 1 if x == '男' else 0)
data7['性别_女'] = data5['性别'].apply(lambda x: 1 if x == '女' else 0)
x = data7['年龄'].values
scaler = MinMaxScaler()
x_normalized = scaler.fit_transform(x.reshape(-1, 1))
binarizer = Binarizer(threshold=0.5)
x_binarized = binarizer.fit_transform(x_normalized)
data7['年龄'] = x_binarized

In [41]:
data7['年龄'].values

array([0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.])

In [42]:
data7

,ID,年龄,性别,脑出血前mRS评分,高血压病史,卒中病史,糖尿病史,房颤史,冠心病史,吸烟史,饮酒史,血压,性别_男,性别_女
0,sub001,0.0,女,0,0,0,0,0,0,0,0,0,0,1
1,sub002,0.0,男,0,1,0,0,0,0,0,0,0,1,0
2,sub003,1.0,男,0,1,0,0,0,0,0,0,0,1,0
3,sub004,1.0,男,2,1,1,0,0,0,0,0,0,1,0
4,sub005,0.0,男,0,0,0,0,0,0,0,0,0,1,0
5,sub006,1.0,男,0,1,0,0,0,0,0,0,0,1,0
6,sub007,0.0,男,0,1,0,0,0,0,0,0,0,1,0
7,sub008,0.0,女,0,1,0,0,0,0,0,0,0,0,1
8,sub009,1.0,女,0,1,1,0,1,0,0,0,0,0,1
9,sub010,1.0,男,0,1,0,0,0,0,1,0,0,1,0


In [43]:
pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns',None)

#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

a=list(data7['脑出血前mRS评分'])
data7['脑出血前mRS评分']=[(i-min(a))/(max(a)-min(a)) for i in a]
data7.drop(columns={'性别'},axis=1,inplace=True)
data7.to_excel('2b数据处理结果.xlsx',index=False)

In [44]:
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, Birch, SpectralClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# 示例数据
data = data7[list(data7.columns)[1:]]


# 定义所需的聚类算法和参数

clustering_algorithms = [
    ("K-Means", KMeans(n_clusters=5)),
    ("Hierarchical Clustering", AgglomerativeClustering(n_clusters=5)),
    ("DBSCAN", DBSCAN(eps=0.5, min_samples=5)),
    ("Birch", Birch(n_clusters=5)),
    ("Spectral Clustering", SpectralClustering(n_clusters=5))
    ]


# 迭代聚类算法并分别计算评估指标
for algorithm_name, algorithm in clustering_algorithms:
    algorithm.fit(data)
    labels = algorithm.labels_

    silhouette = silhouette_score(data, labels)
    ch_index = calinski_harabasz_score(data, labels)

    print("Clustering Algorithm:", algorithm_name)
    print("Clustering Labels:", labels)
    print("Silhouette coefficient:", silhouette)
    print("Calinski-Harabasz index:", ch_index)
    print("---------------------------------------")

Clustering Algorithm: K-Means
Clustering Labels: [2 1 0 0 1 0 1 2 2 0 0 1 1 0 3 2 1 2 2 2 1 0 4 4 1 2 3 0 1 1 1 4 2 1 0 0 2
 1 1 1 0 2 2 1 4 2 1 1 2 2 2 0 4 1 2 2 2 2 0 2 1 1 4 0 2 0 0 1 0 2 2 2 1 1
 1 3 2 4 1 0 2 4 4 4 1 1 4 0 2 1 1 3 4 4 2 2 2 3 1 4]
Silhouette coefficient: 0.3583466251743415
Calinski-Harabasz index: 29.64288543672267
---------------------------------------
Clustering Algorithm: Hierarchical Clustering
Clustering Labels: [0 3 1 1 1 1 3 0 0 2 1 2 1 1 4 0 1 0 0 0 3 1 2 2 3 0 4 1 3 3 3 2 0 3 4 1 0
 3 3 3 1 0 0 3 2 0 3 3 0 0 0 4 3 3 0 0 0 0 1 0 3 3 2 1 0 1 1 2 1 0 0 0 3 3
 3 4 0 2 1 2 0 2 3 2 3 3 2 1 0 3 3 4 2 2 0 0 0 4 3 2]
Silhouette coefficient: 0.33867795396664574
Calinski-Harabasz index: 27.42399095115712
---------------------------------------
Clustering Algorithm: DBSCAN
Clustering Labels: [-1  0  1 -1 -1  1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1  0  1  3 -1
  0 -1 -1  1  0 -1  0  3 -1  0 -1  1  2  0  0  0  1  2 -1  0 -1 -1  0  0
 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 